In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time # 사이트 기동될 때 까지 2초 정도를 줌. - 셀리니움에서 get. request에서 get쓸 때
import pymysql

In [2]:
def vivaUrl(search):
        url_temp = 'http://www.viva100.com/main/search_detail.php?&schLCode=&schMCode=&schSCode=&schBoardCode=&schBoardCode1=&s_word={search}&w_gbn=all&startday=2019-04-01&endday=2019-09-30'.format(search=search)
        driver = webdriver.Chrome("./chromedriver.exe")
        driver.get(url_temp)
        time.sleep(2)
        return vivaUrlList(driver)

In [3]:
# 2단계 목록에서 href 가져오기
def vivaUrlList(driver):
    url_temp = driver.current_url 
    for pageNo in range(1, 25, 1):
        url = url_temp + "&page={pageNo}".format(pageNo=pageNo)
        driver.get(url)
        vivaUrls_List = []
        vivaUrls = driver.find_elements_by_css_selector(".detail_search_box .allnews_list .allnews_tit a")
        if len(vivaUrls) == 0:
            break
        for urlList in vivaUrls:
            vivaUrls_List.append(urlList.get_attribute("href"))
        vivaInfo(driver, vivaUrls_List)

In [4]:
def vivaInfo(driver, vivaUrls_List):
    for urllist in vivaUrls_List:
        driver.get(urllist)
        time.sleep(2)
#         . 은 class, # 은 id값, HTML tag는 그대로 표현됨
        try:
            # 제목
            newsTitleList = driver.find_elements_by_css_selector(".view_top_title")
            newsTitle_temp = newsTitleList[0]
            newsTitle = newsTitle_temp.text

            # 소제목
            newsSubtitle = ""
            newsSubtitleList = driver.find_elements_by_css_selector(".view_top_text")
            newsSubtitle_temp = newsSubtitleList[0]
            newsSubtitle = newsSubtitle_temp.text

            # 날짜
            newsDate = ""
            newsDateList = driver.find_elements_by_css_selector(".view_top_days.view1_top_days")
            newsDate_temp = newsDateList[0]
            newsDate = newsDate_temp.text
            newsDate = newsDate.replace("입력 ", "").replace("-", "")[0:8]

            # 내용
            newsContents = ""
            newsContentsList = driver.find_elements_by_css_selector(".left_text_box")
            for newsContent in newsContentsList:
                newsContents = newsContents + newsContent.text
                
            #  print
            print("url: ", urllist)
            print("제목: ", newsTitle)
            print("소제목: ", newsSubtitle)
#             print("날짜: ", newsDate)
#             print("내용: ", newsContents)

        except:
            print("-----에러 url-----", urllist)
            continue

        dbData = [[urllist, newsTitle, newsDate, newsSubtitle, newsContents]]
        connectDB(dbData)

In [5]:
def connectDB(dbData):
    DB_HOST = '127.0.0.1'
    DB_USER = 'root'
    DB_PASSWD = 'autoset'
    DB_NAME = 'final'
    
    conn = pymysql.connect(host=DB_HOST, user=DB_USER, password=DB_PASSWD,
                          db=DB_NAME, charset='utf8')
    curs = conn.cursor()
    sql = """insert into viva100(url, title, date, sub_title, contents) values (%s, %s, %s, %s, %s)"""
    curs.executemany(sql,dbData)
    conn.commit()
    
    conn.close()

In [6]:
# search = input("검색: ")
search = '일본 여행'
# 보통 우리가 무엇을 검색해서 크롤링 할 지 정해져있기 때문에 미리 저장해둠.
vivaUrl(search)

url:  http://www.viva100.com/main/view.php?key=20191004010001183
제목:  유안타증권 “여행약세 장기화…하나·모두투어 목표주가 하향”
소제목:  
url:  http://www.viva100.com/main/view.php?key=20191002010000715
제목:  카카오, 동남아 모빌리티 최강자 ‘그랩’ 손잡고 베트남 진출
소제목:  
url:  http://www.viva100.com/main/view.php?key=20190930010010009
제목:  소상공인 10명 중 7명, 日 불매운동 피해 겪어
소제목:  
url:  http://www.viva100.com/main/view.php?key=20190930010009897
제목:  [이슈&이슈] 3분기 전국 권역별 경기 ‘정체’
소제목:  
url:  http://www.viva100.com/main/view.php?key=20190930010009824
제목:  중기부, 일본 수출규제 피해 여행·관광업·소상공인에 1100억원 정책자금 지원
소제목:  
url:  http://www.viva100.com/main/view.php?key=20190929010009362
제목:  면세점·카지노株, 외국인 관광객 증가 기대감 고공행진
소제목:  
url:  http://www.viva100.com/main/view.php?key=20190926010008557
제목:  [비바100] 연차내기 눈치 보이는 직장인들을 위한 힐링 ‘짬’내투어 여행지 상하이, 블라디보스토크, 다낭
소제목:  [즐금]주말과 국가 공휴일 활용한 힐링 ‘짬’내투어 3곳
도선미 작가의 중국 상하이, 강한나 작가의 러시아 블라디보스토크, 배나영 작가의 베트남 다낭
‘짧은 여행’ 마니아 직장인 S와 Y의 1석 2조
url:  http://www.viva100.com/main/view.php?key=20190926010008636
제목:  에어서울, 12월 베트남 하노이 신규 취항


url:  http://www.viva100.com/main/view.php?key=20190821010006322
제목:  전남 무안국제공항, 백두산 관문 中 연길 하늘길 열려
소제목:  제주항공, 21일 취항식…매주 수·토요일 운항
url:  http://www.viva100.com/main/view.php?key=20190821010006241
제목:  제주항공, 동남아 노선 예약 급증에 ‘공급석 확대’
소제목:  9~10월 노선 예약자 전년比 59% 증가
url:  http://www.viva100.com/main/view.php?key=20190820010005816
제목:  韓 관광수지 적자, 3년간 350억달러…對일본 적자 3년간 90억달러
소제목:  
url:  http://www.viva100.com/main/view.php?key=20190819001911415
제목:  [인포그래픽] 일본여행 여론조사
소제목:  
url:  http://www.viva100.com/main/view.php?key=20190819010005536
제목:  항공업계 고공비행 제동, 실적 부진 공포
소제목:  2분기 실적 모두 적자전환…“3분기 더 최악일 것” 우려
url:  http://www.viva100.com/main/view.php?key=20190819010005404
제목:  주한베트남관광청, 종로구관광협의회 관광활성화 위한 업무협약
소제목:  
url:  http://www.viva100.com/main/view.php?key=20190819010005323
제목:  일본 경제보복에 상장사 10곳중 6곳 3Q 영업익 전망↓
소제목:  
url:  http://www.viva100.com/main/view.php?key=20190818010005193
제목:  홍콩시위·獨 마이너스성장·금리 하락, 해외발 줄악재에 더 암울해지는 한국경제
소제목:  
url:  http://www.viva100.com/main/view.php?key=20190818010

url:  http://www.viva100.com/main/view.php?key=20190804010000953
제목:  국토부, 상반기 항공이용객 6% 증가…화물은 감소
소제목:  
url:  http://www.viva100.com/main/view.php?key=20190803010000860
제목:  대구시, 일본 수출 규제 관련 총력 대응
소제목:  전략물자품목과 주요 수입품목 비교 분석해 대응방안 모색
url:  http://www.viva100.com/main/view.php?key=20190803010000823
제목:  세종시의회, 日 화이트리스트 한국 제외 규탄 성명서 발표
소제목:  추가 경제 보복 조치 즉각 철회 촉구…일본 제품 등 불매운동 전개
url:  http://www.viva100.com/main/view.php?key=20190803010000807
제목:  세종시의회, 日 화이트리스트 규탄 성명서 발표
소제목:  경제 보복 조치 철회 촉구…일본 제품 불매 운동 전개
url:  http://www.viva100.com/main/view.php?key=20190802010000743
제목:  최대호 안양시장 일본규탄 성명서 발표
소제목:  - 간부 공무원 30여명과 함께 즉각적인 철회요구 -
url:  http://www.viva100.com/main/view.php?key=20190802010000613
제목:  [日 경제보복]日제품 불매운동 장기화 조짐
소제목:  日 경제보복 이후 불매 참여 70% 육박
‘아사히 안 마시고, 유니클로 안 입는다’ 장기화 조짐
url:  http://www.viva100.com/main/view.php?key=20190731010010507
제목:  '2금융권’ 해외송금 시장 경쟁 본격화
소제목:  
url:  http://www.viva100.com/main/view.php?key=20190801010000101
제목:  아시아나항공 인수전 본궤도 오르기도 전 '日 경제보복' 난기류에 '빨

url:  http://www.viva100.com/main/view.php?key=20190708010002924
제목:  [갓 구운 책] 출국에서 입국까지 내 여정과 ‘착! 붙는 여행’ 일본어·영어
소제목:  한동오 ‘착! 붙는 여행 영어’, 박나리 시사일본어학원 교수부장 ‘착! 붙는 여행 일본어’
‘어디서나 써먹는 왕기초 표현’부터 귀국까지를 아우른 ‘착! 붙는 여행 영어’
일본을 여행하면서 꼭 해봐야 할 버킷리스트 30 담은 ‘만끽’ ‘착! 붙는 여행 일본어’
url:  http://www.viva100.com/main/view.php?key=20190708010002797
제목:  ‘일본 불매’ 움직임에 긴장하는 LCC, 중국 신규취항 속도↑
소제목:  
url:  http://www.viva100.com/main/view.php?key=20190708010002725
제목:  난기류 만난 항공업계, 경쟁구도 재편 주목
소제목:  
url:  http://www.viva100.com/main/view.php?key=20190707010002397
제목:  ‘보이콧 일본’ 불매 운동 일파만파...잘 나가던 日 식품유통기업들 ‘긴장’
소제목:  
url:  http://www.viva100.com/main/view.php?key=20190707010002404
제목:  韓‘日제품 불매’ 움직임 주시하는 日…“18일, 대응조치 분기점”
소제목:  아베 “韓, 징용 문제 약속 안 지키는데 대북제재도 안 지킬 수 있어” 주장
url:  http://www.viva100.com/main/view.php?key=20190707010002352
제목:  방일 한국 관광객 8년 만에 감소할 듯...5월까지 4.7% 감소
소제목:  
url:  http://www.viva100.com/main/view.php?key=20190705002355571
제목:  케빈오, 일본 여행 다녀온 인증 사진 공개...‘뜨거운 감자’
소제목:  
url:  http://www.viva

url:  http://www.viva100.com/main/view.php?key=20190606010001697
제목:  티웨이항공 ‘이주의 특가’ 오픈…일본·베트남 특가 운임 제공
소제목:  
url:  http://www.viva100.com/main/view.php?key=20190605010001424
제목:  산청 동의보감촌, 웰니스 관광 한류 중심지 급부상
소제목:  - 터키·이집트·싱가폴 등 방문객 잇따라
- 부산대 우즈벡 의료연수생 한의학 체험
- 남사예담촌·한의학 박물관 등 방문해
url:  http://www.viva100.com/main/view.php?key=20190605010001448
제목:  ‘보복카드’ 모조리 꺼내든 시진핑, 트럼프 대면 앞두고 전방위 압박
소제목:  
url:  http://www.viva100.com/main/view.php?key=20190605010001303
제목:  경상수지 6억6000만달러 적자… 7년 만에 적자 전환
소제목:  
url:  http://www.viva100.com/main/view.php?key=20190604010000951
제목:  아시아나항공, 中·日 등 노선별 퀴즈 이벤트 실시
소제목:  ‘아시아는 아무거나? 아시아도 아시아나!’ 진행
url:  http://www.viva100.com/main/view.php?key=20190603010000471
제목:  한화그룹, 대학생 대상 트래블 콘텐츠 크리에이터 인턴십 모집
소제목:  
url:  http://www.viva100.com/main/view.php?key=20190603010000356
제목:  직장인 올해 휴가비용, 평균 98만원…기간은 '4일'
소제목:  사람인, 휴기 시기는 7월 마지막주
url:  http://www.viva100.com/main/view.php?key=20190603010000370
제목:  티웨이항공, 6월 황금연휴 초특가 행사…대구 출발 일본 노선 대상
소제목:  
url:  http:/

url:  http://www.viva100.com/main/view.php?key=20190425010009125
제목:  근현대사 간직한 개항장 창고 거리…복고 여행 성지로 우뚝
소제목:  인천 공장 골조 그대로 살린 카페가 뉴트로 여행지로 추천
url:  http://www.viva100.com/main/view.php?key=20190424010008952
제목:  청주공항, 중국으로 가는 하늘길 넓어진다!
소제목:  중국 웨이하이 정기노선 첫 취항, -신규 운수권 확보 마중물 역할 기대
url:  http://www.viva100.com/main/view.php?key=20190424010008599
제목:  LF 바네사브루노, ‘카바스 보야지’ 글로벌 팝업스토어 국내 첫 오픈
소제목:  
url:  http://www.viva100.com/main/view.php?key=20190424010008574
제목:  이스타항공, 일본행 항공권 '9900원 특가' 이벤트
소제목:  
url:  http://www.viva100.com/main/view.php?key=20190424010008517
제목:  에어서울, 시즈오카 ‘영원특가’…왕복 5만원대부터
소제목:  
url:  http://www.viva100.com/main/view.php?key=20190423010008154
제목:  글로벌 브랜드 국내 직진출 러쉬…"한국은 아시아 시장 테스트베드"
소제목:  글로벌 브랜드 직진출로 국내 유통·식음료업계 ‘초긴장’
“한국, 아시아서 영향력 크고·글로벌 감각의 소비자층 多”
url:  http://www.viva100.com/main/view.php?key=20190422010007568
제목:  신라면세점, 5월초 日골든위크·中노동절 특수 노린다
소제목:  
url:  http://www.viva100.com/main/view.php?key=20190422010007524
제목:  에어서울, 브이로그 공모전 개최…3명에 ‘1년 무제한 탑승권’ 증정
소